A place to test random stuff

In [1]:
from monkey.model.classification_model.efficientnet_b0 import (
    EfficientNet_B0,
)
import torch
import torch.nn as nn
from torchsummary import summary
from monkey.model.utils import (
    get_classification_metrics,
    get_activation_function,
)
from monkey.model.loss_functions import get_loss_function

/home/u1910100/miniconda3/envs/tiatoolbox/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/u1910100/miniconda3/envs/tiatoolbox/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [7]:
model = EfficientNet_B0(num_classes=2)
model.eval()
# summary(model)
test = torch.randn(size=(4, 3, 32, 32), dtype=torch.float32)
gt = torch.tensor([0, 0, 0, 1], dtype=torch.long)

loss_fn = nn.CrossEntropyLoss()

weighted_ce = get_loss_function("Weighted_CrossEntropy")
weighted_ce.set_weight(torch.tensor([0.4, 0.6]))

pred = []

act = get_activation_function("softmax")

with torch.no_grad():
    out = model(test)
    out = act(out)

    pred_labels = torch.argmax(out, dim=1).cpu().tolist()

    pred.extend(pred_labels)

    loss = loss_fn(out, gt)
    weighted_loss = weighted_ce.compute_loss(out, gt)

gt_labels = gt.cpu().tolist()

print(f"probs {out}")
print(out.shape)
print(f"gt {gt}")
print(gt.shape)
print(f"loss {loss}")
print(f"weighted loss {weighted_loss}")

print(f"pred labels: {pred_labels}")
print(f"true labels: {gt_labels}")

metrics = get_classification_metrics(gt_labels, pred_labels)
print(metrics)

probs tensor([[0.0014, 0.9986],
        [0.0047, 0.9953],
        [0.0032, 0.9968],
        [0.0100, 0.9900]])
torch.Size([4, 2])
gt tensor([0, 0, 0, 1])
torch.Size([4])
loss 1.0612165927886963
weighted loss 0.9787114262580872
pred labels: [1, 1, 1, 1]
true labels: [0, 0, 0, 1]
{'Balanced_Accuracy': 0.5, 'Precision': 0.25, 'Recall': 1.0, 'F1': 0.4}
